In [52]:
import biorefineries
from biorefineries import cellulosic
from biorefineries.ethanol import create_ethanol_purification_system
from biorefineries.tea import create_cellulosic_ethanol_tea
import biosteam as bst
bst.nbtutorial()

In [ ]:
flow (kmol/hr): (l) Water      925
                (s) Sucrose    0.243
                    Ash        1e+03
                    Lignin     46.8
                    Cellulose  215
                    Xylan      78.6
                    Arabinan   1.26
                    Mannan     19
                    Galactan   7.2

In [44]:
class CellulosicEthanolBiorefinery(bst.ProcessModel):
    # The 'Scenario' class defines arguments to the process model.
    # It works very similar to a python dataclass: https://docs.python.org/3/library/dataclasses.html
    class Scenario:
        # To define an argument, you must include the type, but the default is optional.
        # You can also add metadata by tagging along a string that starts with #.
        feedstock: str = 'poplar', '# dry at 20% moisture content'
        pretreatment: str = 'dilute acid'

    @classmethod
    def as_scenario(cls, scenario):
        # Interpret strings in the form of '{feedstock}/{pretreatment}' as a scenario.
        feedstock, pretreatment = scenario.split('/')
        return cls.Scenario(feedstock, pretreatment)

    def create_thermo(self):
        return cellulosic.create_cellulosic_ethanol_chemicals()

    def create_system(self):
        # Here we create the units and connect them.
        # BioSTEAM can take care of creating the system.
        cellulosic.load_process_settings()
        scenario = self.scenario # The input parameters to the process model are saved here.
        if self.scenario.feedstock == 'poplar':
            feedstock = bst.Stream(
                ID='feedstock',
                price=0,
                total_flow=1e+05,
                units='kg/hr',
                Water=0.002880605,
                Sucrose=0.00143568,
                Extract=0,
                Acetate=0,
                Ash=0.017248096,
                Lignin=0.12263097,
                Protein=0.0,
                Glucan=0.600193853,
                Xylan=0.17945995,
                Arabinan=0.002871359,
                Mannan=0.053140087,
                Galactan=0.020139395,
            )
        elif self.scenario.feedstock == 'switchgrass':
            feedstock = bst.Stream(
                ID='feedstock',
                total_flow=104229.16,
                price=0.08,
                units='kg/hr',
                Arabinan=0.02789,
                Galactan=0.01044,
                Glucan=0.2717,
                Xylan=0.21215,
                Mannan=0.00594,
                Lignin=0.17112,
                Ash=0.01619,
                Extract=0.0756,
                Acetate=0.00897,
                Water=0.2,
            )
        else:
            raise ValueError('invalid feedstock')

        ethanol = bst.Stream(ID='ethanol', price=0.5524509505703422)
        U101 = cellulosic.units.FeedStockHandling('U101', feedstock)
        U101.cost_items['System'].cost = 0.

        if scenario.pretreatment == 'dilute acid':
            create_pretreatment_sys = cellulosic.create_dilute_acid_pretreatment_system
        elif scenario.pretreatment == 'AFEX':
            create_pretreatment_sys = cellulosic.create_ammonia_fiber_expansion_pretreatment_system
        else:
            raise ValueError('invalid pretreatment')

        pretreatment_sys = create_pretreatment_sys(
            ins=U101-0, area=200, mockup=True,
        )
        fermentation_sys = cellulosic.create_cellulosic_fermentation_system(
            ins=pretreatment_sys.get_outlet('pretreated_biomass'), area=300, mockup=True,
            # Valid arguments include:
            # Integrated Bioprocess (IB)
            # Simultaneous Saccharification and Co-Fermentation (SSCF)
            # Saccharification and Co-Fermentation (SCF)
            kind='IB',
        )
        ethanol_purification_sys = create_ethanol_purification_system(
            ins=fermentation_sys.get_outlet('beer'),
            outs=[ethanol], area=400, mockup=True,
        )
        ethanol, stillage, stripper_bottoms_product = ethanol_purification_sys.outs
        water = bst.Stream(Water=1, T=47+273.15, P=3.9*101325, units='kg/hr')
        S401 = bst.PressureFilter(400, (stillage, water))
        bst.create_all_facilities(
            # Certain facilities like the Fire Water Tank (in case there is a fire)
            # is sized based on feedstock flow rate
            feedstock,
            recycle_process_water_streams=[stripper_bottoms_product],
            HXN=False, # No heat exchanger network,
            area=600,
        )


    def create_model(self): # We create the Model object here.
        system = self.system # BioSTEAM automaticaly creates the system and saves it as self.system
        self.tea = tea = create_cellulosic_ethanol_tea(system)
        model = bst.Model(system)
        price = self.feedstock.price * 1000 # USD / MT
        processing_capacity = self.feedstock.F_mass * tea.operating_hours / 1e6 # 10^3 MT / y

        @model.parameter(
            element='feedstock', units='USD/kg', # Metadata for pretty tables/plots
            baseline=price, # Baseline price
            bounds=(price * 0.9, price * 1.1) # Min/max price
        )
        def set_feedstock_price(feedstock_price):
            self.feedstock.price = feedstock_price / 1000

        @model.parameter(
            element='feedstock', units='10^3 MT/y',
            baseline=processing_capacity,
            bounds=(processing_capacity * 0.9, processing_capacity * 1.1)
        )
        def set_processing_capacity(processing_capacity):
            self.feedstock.F_mass = 1e6 * processing_capacity /  tea.operating_hours

        @model.indicator(units='USD/gal')
        def MESP():
            return tea.solve_price(self.ethanol) * 2.98668849 # USD/kg to USD/gal

        return model

In [45]:
cellulosic_br = CellulosicEthanolBiorefinery(scenario='poplar/dilute acid', simulate=False)
cellulosic_br

CellulosicEthanolBiorefinery(
    # dry at 20% moisture content
    feedstock='poplar',
    pretreatment='dilute acid',
)


In [46]:
cellulosic_br.system.simulate()


c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\units\design_tools\pressure_vessel.py:104: CostWarning: <RefluxDrum: reflux_drum> Vertical vessel length (9 ft) is out of bounds (12 to 40 ft) for cost correlation
  return method(pressure, diameter, length)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\utils\registry.py:103: RuntimeWarning: upon renaming, <System: SYS43> replaced <System: SYS38> in registry
  warn(warning, 4)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\utils\registry.py:103: RuntimeWarning: upon renaming, <System: SYS44> replaced <System: SYS39> in registry
  warn(warning, 4)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\utils\registry.py:103: RuntimeWarning: upon renaming, <System: SYS45> replaced <System: SYS40> in registry
  warn(warning, 4)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\utils\registry.py:103: RuntimeWarning: upon renaming, <System: SYS46> replaced <Sy

In [65]:
popla = bst.Stream(Water = 100)

In [ ]:
cellulosic_br.system.ins[0] 

Stream: feedstock to <FeedStockHandling: U101>
phase: 'l', T: 298.15 K, P: 101325 Pa
flow (kmol/hr): Water     16
                Sucrose   0.419
                Ash       1.72e+03
                Lignin    80.6
                Glucan    370
                Xylan     136
                Arabinan  2.17
                ...       45.2


In [66]:
cellulosic_br.system.ins[0]= popla

In [69]:
cellulosic_br.system.simulate()


In [70]:
cellulosic_br.set_feedstock_price(0)
cellulosic_br.MESP()

1.326581207206779

In [68]:
cellulosic_br.system.diagram()

In [54]:
from biosteam import main_flowsheet as F, settings, units


In [ ]:
cellulosic_br.system.rescale(feedstock: 'poplar', 1.5)

SyntaxError: positional argument follows keyword argument (3559756929.py, line 1)

In [ ]:
cellulosic_br.system.units.('M608'


SyntaxError: invalid syntax (647725070.py, line 1)

In [49]:
cellulosic_br.set_feedstock_price(0)
cellulosic_br.MESP()

1.3265786399231403

In [8]:
cellulosic_br.system.units

[<FeedStockHandling: U101>,
 <SulfuricAcidStorageTank: T201>,
 <SulfuricAcidTank: T202>,
 <SulfuricAcidMixer: M201>,
 <SteamMixer: M202>,
 <PretreatmentReactorSystem: R201>,
 <BlowdownDischargePump: P201>,
 <OligomerConversionTank: T203>,
 <PretreatmentFlash: F201>,
 <AmmoniaStorageTank: T204>,
 <AmmoniaMixer: M204>,
 <AmmoniaAdditionTank: T205>,
 <HydrolyzatePump: P202>,
 <EnzymeHydrolysateMixer: M301>,
 <HydrolysateHeatExchanger: H301>,
 <ContinuousPresaccharification: U301>,
 <CSLStorageTank: T302>,
 <MockSplitter: S302>,
 <DAPStorageTank: T301>,
 <MockSplitter: S301>,
 <SaccharificationAndCoFermentation: R301>,
 <SeedTrain: U302>,
 <SeedHoldTank: T303>,
 <Mixer: M303>,
 <VentScrubber: U303>,
 <Mixer: M302>,
 <BeerTank: T304>,
 <Pump: P401>,
 <HXprocess: H401>,
 <BinaryDistillation: D401>,
 <Pump: P402>,
 <PressureFilter: U402>,
 <Mixer: M203>,
 <WasteVaporCondenser: H201>,
 <Mixer: M601>,
 <WastewaterSystemCost: U601>,
 <AnaerobicDigestion: R601>,
 <Mixer: M602>,
 <AerobicDigestion

In [9]:
cellulosic_br.system.show()

System: SYS10
Highest convergence error among components in recycle
stream M604-0 after 1 loops:
- flow rate   8.16e+00 kmol/hr (0.025%)
- temperature 4.64e-06 K (1.5e-06%)
ins...
[0] feedstock  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water    1.17e+03
                    Sucrose  1.9
                    Extract  68.5
                    Acetate  25.4
                    Ash      4.15e+03
                    Lignin   87.3
                    Protein  114
                    ...      326
[1] warm_process_water_1  
    phase: 'l', T: 368.15 K, P: 476228 Pa
    flow (kmol/hr): Water  1.3e+03
[2] sulfuric_acid  
    phase: 'l', T: 294.15 K, P: 547155 Pa
    flow (kmol/hr): Water  7.2
                    H2SO4  18.3
[3] pretreatment_steam  
    phase: 'g', T: 541.15 K, P: 1.31722e+06 Pa
    flow (kmol/hr): Water  1.81e+03
[4] warm_process_water_2  
    phase: 'l', T: 368.15 K, P: 476228 Pa
    flow (kmol/hr): Water  8.52e+03
[5] ammonia_process_water  
    phase: 'l',

In [ ]:
cellulosic_br.system.outs[1].price 

0.5

In [42]:
def ethanol_price_converter(price):
    '''
    Function to convert the price of ethanol from USD/gal to USD/kg as BioSTEAM takes in values in USD/kg
    '''
    updated_price = (price*264.172)/789  # 789 is the density of ethanol that is at 20 C from Aspen Plus, a value taken from 2011 Humbird report
    return updated_price

In [43]:
ethanol_price_converter(1.65)

0.5524509505703422

In [17]:
cellulosic_br.system.feeds[3]

Stream: pretreatment_steam to <SteamMixer: M202>
phase: 'g', T: 541.15 K, P: 1.31722e+06 Pa
flow (kmol/hr): Water  1.81e+03


In [18]:
pretty_sys = cellulosic.create_dilute_acid_pretreatment_system

In [26]:
pretty_sys.outs[0]

{'ID': 'pretreated_biomass'}